In [ ]:
!pip install transformers torch
!pip install transformers torch pandas


In [ ]:
!pip install pytorch-forecasting pytorch-lightning


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 3.7 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()  # Opens a file chooser to select files


Saving soil_moisture_Barrow_2012_2013.csv to soil_moisture_Barrow_2012_2013.csv
Saving flux_CO2_CH4_Barrow_2012_2013.csv to flux_CO2_CH4_Barrow_2012_2013.csv


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import wandb
from sklearn.preprocessing import StandardScaler

# Initialize WandB
wandb.init(project="soil-moisture-forecast", name="lstm-forecast")

# Load datasets
soil_moisture_data = pd.read_csv("soil_moisture_Barrow_2012_2013.csv")
flux_data = pd.read_csv("flux_CO2_CH4_Barrow_2012_2013.csv")

# Merge datasets on common columns
merged_data = pd.merge(soil_moisture_data, flux_data, on=["date", "plot_ID"], suffixes=('_moisture', '_flux'))
merged_data["date"] = pd.to_datetime(merged_data["date"], errors="coerce")
merged_data = merged_data.dropna(subset=["date", "Ka", "VWC", "flux_CO2", "flux_CH4"])

# Ensure numeric types for all relevant columns
numeric_cols = ["Ka", "VWC", "flux_CO2", "flux_CH4"]
merged_data[numeric_cols] = merged_data[numeric_cols].apply(pd.to_numeric, errors='coerce')
merged_data = merged_data.dropna(subset=numeric_cols)

# Sort and reset index
merged_data = merged_data.sort_values(["plot_ID", "date"]).reset_index(drop=True)

# Feature scaling for model input
scaler = StandardScaler()
merged_data[["Ka", "VWC", "flux_CH4"]] = scaler.fit_transform(merged_data[["Ka", "VWC", "flux_CH4"]])

# Define a PyTorch Dataset class
class TimeSeriesDataset(Dataset):
    def __init__(self, data, sequence_length=15):
        self.data = data
        self.sequence_length = sequence_length

    def __len__(self):
        return len(self.data) - self.sequence_length

    def __getitem__(self, idx):
        x = self.data[["Ka", "VWC", "flux_CH4"]].iloc[idx:idx + self.sequence_length].values
        y = self.data["flux_CO2"].iloc[idx + self.sequence_length]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

# Create Dataset and DataLoader
sequence_length = 15
dataset = TimeSeriesDataset(merged_data, sequence_length=sequence_length)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Define LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, input_size=3, hidden_size=50, output_size=1, num_layers=2):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        _, (h_n, _) = self.lstm(x)
        return self.fc(h_n[-1])

# Initialize model, loss function, and optimizer
model = LSTMModel()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Training loop
epochs = 10
for epoch in range(epochs):
    model.train()
    epoch_loss = 0.0

    for x, y in dataloader:
        optimizer.zero_grad()
        output = model(x)
        loss = criterion(output.squeeze(), y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(dataloader)
    wandb.log({"epoch": epoch, "loss": avg_loss})
    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

wandb.finish()
print("Training completed.")


<ipython-input-52-8b4ec39a6eef>:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  merged_data["date"] = pd.to_datetime(merged_data["date"], errors="coerce")


Epoch 1/10, Loss: 1.1666
Epoch 2/10, Loss: 1.1443
Epoch 3/10, Loss: 1.1227
Epoch 4/10, Loss: 1.1014
Epoch 5/10, Loss: 1.0802
Epoch 6/10, Loss: 1.0588
Epoch 7/10, Loss: 1.0371
Epoch 8/10, Loss: 1.0149
Epoch 9/10, Loss: 0.9921
Epoch 10/10, Loss: 0.9687


epoch,▁▂▃▃▄▅▆▆▇█
loss,█▇▆▆▅▄▃▃▂▁
epoch,9
loss,0.96865


Training completed.


In [ ]:
import wandb

sweep_config = {
    'method': 'bayes',  # Bayesian search, can be 'grid' or 'random' as well
    'metric': {
        'name': 'loss',
        'goal': 'minimize'
    },
    'parameters': {
        'learning_rate': {
            'values': [1e-2, 1e-3, 1e-4]
        },
        'batch_size': {
            'values': [32, 64, 128]
        },
        'sequence_length': {
            'values': [15, 25, 50]
        },
        'hidden_size': {
            'values': [64, 128, 256]
        },
        'num_layers': {
            'values': [1, 2, 3]
        }
    }
}
sweep_id = wandb.sweep(sweep_config, project="soil-moisture-forecast")


Create sweep with ID: bn8plzvp
Sweep URL: https://wandb.ai/shreyaskaps/soil-moisture-forecast/sweeps/bn8plzvp


In [ ]:
import torch
from torch.utils.data import Dataset

class SoilMoistureDataset(Dataset):
    def __init__(self, data, sequence_length=15):
        self.data = data
        self.sequence_length = sequence_length

    def __len__(self):
        return len(self.data) - self.sequence_length

    def __getitem__(self, idx):
        # Get a sequence of features for the input (Ka, VWC, flux_CH4) and the target (flux_CO2)
        x = self.data[["Ka", "VWC", "flux_CH4"]].iloc[idx:idx + self.sequence_length].values
        y = self.data["flux_CO2"].iloc[idx + self.sequence_length]

        # Convert to torch tensors
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)

        return x, y


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

def train():
    # Initialize a new W&B run
    with wandb.init() as run:
        config = wandb.config

        # Use the hyperparameters from W&B
        learning_rate = config.learning_rate
        batch_size = config.batch_size
        sequence_length = config.sequence_length
        hidden_size = config.hidden_size
        num_layers = config.num_layers

        # Define your dataset and dataloader
        dataset = SoilMoistureDataset(merged_data, sequence_length=sequence_length)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

        # Define your model, loss function, and optimizer
        model = LSTMModel(input_size=3, hidden_size=hidden_size, num_layers=num_layers)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        # Training loop
        for epoch in range(5):  # Adjust epochs if necessary
            model.train()
            epoch_loss = 0.0

            for x, y in dataloader:
                optimizer.zero_grad()
                output = model(x)
                loss = criterion(output, y)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()

            avg_loss = epoch_loss / len(dataloader)
            wandb.log({"epoch": epoch, "loss": avg_loss})



In [ ]:
wandb.agent(sweep_id, train, count=20)  # `count` sets the number of total trials


wandb: Agent Starting Run: cdnaolw6 with config:
wandb: 	batch_size: 32
wandb: 	hidden_size: 256
wandb: 	learning_rate: 0.001
wandb: 	num_layers: 3
wandb: 	sequence_length: 15


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([7])) that is different to the input size (torch.Size([7, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, t

epoch,▁▃▅▆█
loss,▇▄▄█▁
epoch,4
loss,0.67654


wandb: Agent Starting Run: wdmi2gij with config:
wandb: 	batch_size: 32
wandb: 	hidden_size: 128
wandb: 	learning_rate: 0.0001
wandb: 	num_layers: 2
wandb: 	sequence_length: 25


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([29])) that is different to the input size (torch.Size([29, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([29])) that is different to the input size (torch.Size([29, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([29])) that is different to the input size (torch.Size([29, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input,

...

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([39])) that is different to the input size (torch.Size([39, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([39])) that is different to the input size (torch.Size([39, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([39])) that is different to the input size (torch.Size([39, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input,

epoch,▁▃▅▆█
loss,█▆▄▃▁
epoch,4
loss,1.40147


wandb: Agent Starting Run: y40iupq1 with config:
wandb: 	batch_size: 64
wandb: 	hidden_size: 64
wandb: 	learning_rate: 0.001
wandb: 	num_layers: 2
wandb: 	sequence_length: 50


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, targe

epoch,▁▃▅▆█
loss,█▆▄▃▁
epoch,4
loss,1.31528


In [ ]:
!pip install git+https://github.com/amazon-science/chronos-forecasting.git


  Cloning https://github.com/amazon-science/chronos-forecasting.git to /tmp/pip-req-build-05cggs6n
  Running command git clone --filter=blob:none --quiet https://github.com/amazon-science/chronos-forecasting.git /tmp/pip-req-build-05cggs6n
  Resolved https://github.com/amazon-science/chronos-forecasting.git to commit ac6ee36acee1e47446cd66f72f540c87f1f1fbe4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for chronos: filename=chronos-1.2.1-py3-none-any.whl size=13927 sha256=5fc92020f203354f29b0a9853e8a797047c77b8e6526633491a5214531c1f110
  Stored in directory: /tmp/pip-ephem-wheel-cache-jrgtk_hi/wheels/bf/c1/65/08857e57345ef1b34ba0edb4791f1b2594943e82f34e93a4ab
Successfully built chronos


In [ ]:

!rm -rf ~/.cache/huggingface/hub/*


